In [ ]:
import os

import cdsapi
import xarray as xr

_ = xr.set_options(keep_attrs=True)
os.chdir(os.getenv("DATA_VOLUME", "../../../.."))
!pwd

In [ ]:
ecv_store = "s3://edh-root/more-era5/ecv-for-climate-change-1981-2010-anomaly-v0/"
ds = xr.open_dataset(
    ecv_store,
    engine="zarr",
)
ds

In [ ]:
dataset = "ecv-for-climate-change"
base_request = {
        'format': 'zip',
        # 'variable': 'surface_air_temperature',
        'product_type': 'anomaly',
        'climate_reference_period': '1981_2010',
        'time_aggregation': '1_month_mean',
        # 'year': ['2023'],
        # 'month': ['01'],
        'origin': 'era5',
    }

In [ ]:
year_month = "2025-09"

In [ ]:
try:
    ds.sel(valid_time=year_month)
except KeyError:
    print("OK, data not present")
else:
    raise ValueError("STOP, the data is already present!")

print("LAST DATA IS", str(ds.valid_time.values[-1])[:7], "and you are adding", year_month)

In [ ]:
year, _, month = year_month.partition("-")
year_month_request = base_request | {"year": [year], "month": [month]}
year_month_request

In [ ]:
client = cdsapi.Client()

In [ ]:
ecv_0 = [
    "surface_air_temperature",
    "surface_air_relative_humidity",
    "sea_ice_cover"
]
target_0 = f"ecv-0-{year_month}"
target_0

In [ ]:
result = client.retrieve(dataset, year_month_request | {"variable": ecv_0}, target=target_0 + ".zip")
!unzip -o {target_0}.zip -d {target_0}

In [ ]:
ds_ecv_0 = xr.open_mfdataset(f"{target_0}/*grib", time_dims=["valid_time"], squeeze=False).isel(number=0, surface=0)
ds_ecv_0

In [ ]:
ecv_6 = ["precipitation"]
target_6 = f"ecv-6-{year_month}"
target_6

In [ ]:
result = client.retrieve(dataset, year_month_request | {"variable": ecv_6}, target=target_6 + ".zip")
!unzip {target_6}.zip -d {target_6}

In [ ]:
ds_ecv_6 = xr.open_mfdataset(f"{target_6}/*grib", time_dims=["valid_time"], squeeze=False).isel(number=0, surface=0)
ds_ecv_6 = ds_ecv_6.assign_coords(valid_time=ds_ecv_0.valid_time)
ds_ecv_6

In [ ]:
ds_ecv = xr.merge([ds_ecv_0, ds_ecv_6]).drop_vars(["number", "surface"])
ds_ecv

In [ ]:
#
# uncommento to UPDATE PRODUCTION!
#

ds_ecv.to_zarr(ecv_store, mode="a", append_dim="valid_time")

In [ ]:
!aws s3 ls